<a href="https://colab.research.google.com/github/Vsingh31/Amazon_Vine_Analysis/blob/main/Vine_Review_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [73.9 kB]
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [696 B]
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:9 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:11 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:12 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:13 https://developer.download.nvi

In [3]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2022-01-03 03:41:28--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  1.22MB/s    in 0.8s    

2022-01-03 03:41:30 (1.22 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [4]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [5]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Video_Games_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   12039526| RTIS3L2M1F5SM|B001CXYMFS|     737716809|Thrustmaster T-Fl...|     Video Games|          5|            0|          0|   N|                Y|an amazing joysti...|Used this for Eli...| 2015-08-31|
|         US|    9636577| R1ZV7R40OLHKD|B00M920ND6|     569686175|Tonsee 6 buttons ...|     Video Games|          5|    

In [6]:
#Filter the data and create a new DataFrame or table to retrieve all the rows where the total_votes count is equal to or greater 
#than 20 to pick reviews that are more likely to be helpful and to avoid having division by zero errors later on.
# Create the vine_table. DataFrame
vine_df = df.select(["review_id","star_rating","helpful_votes","total_votes","vine","verified_purchase"])
vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| RTIS3L2M1F5SM|          5|            0|          0|   N|                Y|
| R1ZV7R40OLHKD|          5|            0|          0|   N|                Y|
|R3BH071QLH8QMC|          1|            0|          1|   N|                Y|
|R127K9NTSXA2YH|          3|            0|          0|   N|                Y|
|R32ZWUXDJPW27Q|          4|            0|          0|   N|                Y|
|R3AQQ4YUKJWBA6|          1|            0|          0|   N|                Y|
|R2F0POU5K6F73F|          5|            0|          0|   N|                Y|
|R3VNR804HYSMR6|          5|            0|          0|   N|                Y|
| R3GZTM72WA2QH|          5|            0|          0|   N|                Y|
| RNQOY62705W1K|          4|            0|          0|   N|     

In [7]:
#Filter the new DataFrame or table created in Step 1 and create a new DataFrame or table to retrieve all the rows 
# where the number of helpful_votes divided by total_votes is equal to or greater than 50%.
No_of_helpfulvotes_df = vine_df.filter((vine_df.helpful_votes  / vine_df.total_votes) >= 0.5 )
No_of_helpfulvotes_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R23H79DHOZTYAU|          1|            1|          1|   N|                Y|
|R3AACSKX1EAHUY|          5|            1|          1|   N|                Y|
|R1FJWU0IJ4KUT4|          1|            5|          5|   N|                Y|
|R23HC6NMA3HBPS|          5|            1|          1|   N|                N|
| R8HD4KOH8L0A6|          5|            1|          2|   N|                Y|
|R3DCTXDTB028VH|          5|            3|          4|   N|                Y|
| R4PKAZRQJJX14|          1|           21|         34|   N|                N|
|R2NCZ6CIHAWH28|          5|            1|          1|   N|                Y|
|R2CI0Y288CC7E2|          1|           21|         35|   N|                Y|
| RXJ4RXVZNQQMK|          1|            1|          1|   N|     

In [8]:
#Filter the DataFrame or table created in Step 2, and create a new DataFrame or table that retrieves
# all the rows where a review was written as part of the Vine program (paid), vine == 'Y'.
vine_yes_df = No_of_helpfulvotes_df.filter(No_of_helpfulvotes_df.vine == 'Y')
vine_yes_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| RWKQQPG100CQ4|          4|            2|          4|   Y|                N|
|R1R0J8S1VGALUY|          1|            1|          2|   Y|                N|
|R28165EG0N9CSN|          4|            1|          2|   Y|                N|
|R1CH0ZXGIMVEFC|          5|            1|          2|   Y|                N|
|R2BFEJHEKH4YWZ|          5|            3|          4|   Y|                N|
| RYIG0BYEZP7WM|          2|            1|          1|   Y|                N|
|R20FZQEZM1277P|          4|            2|          4|   Y|                N|
| RSAY2UB0IHC0L|          4|            4|          6|   Y|                N|
| RQKQVDDOKW5MU|          5|            4|          5|   Y|                N|
|R379LKZSA2PFOZ|          4|            1|          2|   Y|     

In [9]:
#Repeat Step 3, but this time retrieve all the rows where the review was not part of the Vine program (unpaid), vine == 'N'.
vine_No_df = No_of_helpfulvotes_df.filter(No_of_helpfulvotes_df.vine == 'N')
vine_No_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R23H79DHOZTYAU|          1|            1|          1|   N|                Y|
|R3AACSKX1EAHUY|          5|            1|          1|   N|                Y|
|R1FJWU0IJ4KUT4|          1|            5|          5|   N|                Y|
|R23HC6NMA3HBPS|          5|            1|          1|   N|                N|
| R8HD4KOH8L0A6|          5|            1|          2|   N|                Y|
|R3DCTXDTB028VH|          5|            3|          4|   N|                Y|
| R4PKAZRQJJX14|          1|           21|         34|   N|                N|
|R2NCZ6CIHAWH28|          5|            1|          1|   N|                Y|
|R2CI0Y288CC7E2|          1|           21|         35|   N|                Y|
| RXJ4RXVZNQQMK|          1|            1|          1|   N|     

In [10]:
#Determine the total number of reviews, the number of 5-star reviews, and the percentage of 5-star reviews for the two types of review (paid vs unpaid).
Total_noof_review_df= vine_df.agg({"review_id" : "count"})
Total_noof_review_df.show()


+----------------+
|count(review_id)|
+----------------+
|         1785997|
+----------------+



In [37]:
# Determine the number of 5-star reviews.
fivestar = vine_df.filter((vine_df.star_rating) == 5)
total_fivestarreview= fivestar.agg({"star_rating" : "count"}).withColumnRenamed("count(star_rating)", "Total_fivestar_rating")
total_fivestarreview.show()

+---------------------+
|Total_fivestar_rating|
+---------------------+
|              1026924|
+---------------------+



In [42]:
# Determine the percentage of 5-star reviews for the two types of review (paid vs unpaid)

fivestar_for_paid = vine_df.filter(((vine_df.star_rating) == 5) & ((vine_df.vine) == 'Y'))
total_fivestar_for_paid= fivestar_for_paid.agg({"star_rating" : "count"}).withColumnRenamed("count(star_rating)", "Total_fivestar_paid_rating")
total_fivestar_for_paid.show()
percentage_yes_fivestarreview = (1607 / 1026924) * 100
percentage_yes_fivestarreview

+--------------------------+
|Total_fivestar_paid_rating|
+--------------------------+
|                      1607|
+--------------------------+



0.15648675072351995

In [43]:
# Determine the percentage of 5-star reviews for the two types of review (paid vs unpaid)

fivestar_for_unpaid = vine_df.filter(((vine_df.star_rating) == 5) & ((vine_df.vine) == 'N'))
total_fivestar_for_unpaid= fivestar_for_unpaid.agg({"star_rating" : "count"}).withColumnRenamed("count(star_rating)", "Total_fivestar_Unpaid_rating")
total_fivestar_for_unpaid.show()
percentage_No_fivestarreview = (1025317 / 1026924) * 100
percentage_No_fivestarreview

+----------------------------+
|Total_fivestar_Unpaid_rating|
+----------------------------+
|                     1025317|
+----------------------------+



99.84351324927648